GAN (Generative Adversarial Network)
- https://github.com/taki0112/GAN-Tensorflow
- https://github.com/zalandoresearch/fashion-mnist

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from skimage.io import imsave
import os
import shutil
import gzip
img_height = 28
img_width = 28

img_size = img_height * img_width

to_train = True
to_restore = False
output_path = "samples"

max_epoch = 500

h1_size = 150
h2_size = 300
z_size = 100
batch_size = 256


In [2]:
def load_mnist(path, kind='train'):

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
def generator(z):
    h1 = tf.layers.dense(inputs=z, units=h1_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(inputs=h1, units=h2_size, activation=tf.nn.relu)
    return tf.layers.dense(inputs=h2, units=img_size, activation=tf.nn.tanh)

In [4]:
def discriminator(x_data, x_generated, keep_prob):
    x_in = tf.concat([x_data, x_generated],0)

    h1 = tf.nn.dropout(tf.layers.dense(inputs=x_in, units=h2_size, activation=tf.nn.relu),
                       keep_prob=keep_prob)
    h2 = tf.nn.dropout(tf.layers.dense(inputs=h1, units=h1_size, activation=tf.nn.relu),
                       keep_prob=keep_prob)
    h3 = tf.layers.dense(inputs=h2, units=1, activation=tf.nn.relu)
    
    h3_data = tf.slice(h3, [0,0], [batch_size, -1], name=None)
    h3_generated = tf.slice(h3, [batch_size, 0], [-1,-1], name=None)

    y_data = tf.nn.sigmoid(h3_data)
    y_generated = tf.nn.sigmoid(h3_generated)

    return y_data, y_generated

In [5]:
def show_result(batch_res, fname, grid_size=(8, 8), grid_pad=5):
    batch_res = 0.5 * batch_res.reshape((batch_res.shape[0], img_height, img_width)) + 0.5
    img_h, img_w = batch_res.shape[1], batch_res.shape[2]
    grid_h = img_h * grid_size[0] + grid_pad * (grid_size[0] - 1)
    grid_w = img_w * grid_size[1] + grid_pad * (grid_size[1] - 1)
    img_grid = np.zeros((grid_h, grid_w), dtype=np.uint8)
    for i, res in enumerate(batch_res):
        if i >= grid_size[0] * grid_size[1]:
            break
        img = (res) * 255.
        img = img.astype(np.uint8) 
        row = (i // grid_size[0]) * (img_h + grid_pad)
        col = (i % grid_size[1]) * (img_w + grid_pad)
        img_grid[row:row + img_h, col:col + img_w] = img
    imsave(fname, img_grid)

In [6]:
def train():    
    global_step = tf.Variable(0, name="global_step", trainable=False)
    
    train_x, train_y = load_mnist("fashion_mnist", kind='train')
    size = train_x.shape[0]
    x_data = tf.placeholder(tf.float32, [batch_size, img_size], name = "x_data")
    z = tf.placeholder(tf.float32, [batch_size, z_size], name = 'z')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    
    with tf.variable_scope('generator'):
        x_generated = generator(z)
    with tf.variable_scope('discriminator'):
        y_data, y_generated = discriminator(x_data, x_generated, keep_prob)
    
    d_loss = - (tf.log(y_data) + tf.log(1 - y_generated))
    g_loss = - tf.log(y_generated)
    
    d_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
    g_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
    #d_params = filter(lambda x: x.name.startswith('discriminator'),
    #                  tf.trainable_variables())
    #g_params = filter(lambda x: x.name.startswith('generator'),
    #                  tf.trainable_variables())

    optimizer = tf.train.AdamOptimizer(0.0001)

    d_trainer = optimizer.minimize(d_loss, var_list=d_params)
    g_trainer = optimizer.minimize(g_loss, var_list=g_params)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    sess = tf.Session()
    sess.run(init)

    if to_restore:
        chkpt_fname = tf.train.latest_checkpoint(output_path)
        saver.restore(sess, chkpt_fname)
    else:
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.mkdir(output_path)

    z_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)

    for i in range(sess.run(global_step), max_epoch):
        if i % 50 == 0:
            print("epoch:%s" % (i))

        for j in range(21870 // batch_size):

            batch_end = j * batch_size + batch_size
            if batch_end >= size:
                batch_end = size - 1
            x_value = train_x[ j * batch_size : batch_end ]
            x_value = x_value / 255.
            x_value = 2 * x_value - 1

            z_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
            sess.run(d_trainer,
                     feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
            if j % 1 == 0:
                sess.run(g_trainer,
                         feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
        x_gen_val = sess.run(x_generated, feed_dict={z: z_sample_val})
        if i % 10 == 0 or i == max_epoch-1:
            show_result(x_gen_val, os.path.join(output_path, "sample%s.jpg" % i))
        sess.run(tf.assign(global_step, i + 1))

In [7]:
#max_epoch = 200
train()

epoch:0
epoch:50
epoch:100
epoch:150
epoch:200
epoch:250
epoch:300
epoch:350
epoch:400
epoch:450
